In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import zipfile

In [ ]:
# !wget 'https://anaconda.org/conda-forge/libjpeg-turbo/2.1.0/download/linux-64/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -q
# !wget 'https://anaconda.org/conda-forge/libgcc-ng/9.3.0/download/linux-64/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -q
# !wget 'https://anaconda.org/conda-forge/gdcm/2.8.9/download/linux-64/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -q
# !wget 'https://anaconda.org/conda-forge/conda/4.10.1/download/linux-64/conda-4.10.1-py37h89c1867_0.tar.bz2' -q
# !wget 'https://anaconda.org/conda-forge/certifi/2020.12.5/download/linux-64/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -q
# !wget 'https://anaconda.org/conda-forge/openssl/1.1.1k/download/linux-64/openssl-1.1.1k-h7f98852_0.tar.bz2' -q

In [ ]:
# !conda install 'libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
# !conda install 'libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
# !conda install 'gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
# !conda install 'conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
# !conda install 'certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
# !conda install 'openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y
# import gdcm
!conda install gdcm -c conda-forge -y

In [ ]:
import pydicom
import cv2
from pydicom.pixel_data_handlers.util import apply_voi_lut

def read_xray(path, voi_lut = True, fix_monochrome = True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
    return data

In [ ]:
dir_path_out = r'train_png'
os.makedirs(dir_path_out)

In [ ]:
import tqdm

In [ ]:
list_imgs = []
dir_path = r'../input/siim-covid19-detection/train'
for dirs, subdirs, filenames in tqdm.tqdm(os.walk(dir_path)):
    for i_file in filenames:
        path_full = os.path.join(dirs, i_file)
        list_imgs.append(path_full)

In [ ]:
len(list_imgs)
zf = zipfile.ZipFile(os.path.join(dir_path_out, 'train_3.zip'), mode='w')

In [ ]:
# for dirs, subdirs, filenames in tqdm.tqdm(os.walk(dir_path)):
for idx, img_path in tqdm.tqdm(enumerate(list_imgs), total=len(list_imgs)):
    
    if idx < 5000:
        continue
        
#     if idx > 5000:
#         break
    
    path_out = os.path.join(dir_path_out, os.path.relpath(img_path, dir_path))
    path_out = os.path.splitext(path_out)[0] + '.png'
    if not os.path.exists(path_out):
        if not os.path.exists(os.path.split(path_out)[0]):
            os.makedirs(os.path.split(path_out)[0])
    img = read_xray(img_path)
    if img is not None:
        cv2.imwrite(path_out, img)
        zf.write(path_out)
        os.remove(path_out)

In [ ]:
# import matplotlib.pyplot as plt

# img = cv2.imread(r'./train_png/a33697e797cc/69ccac342782/014a736182d3.tiff', cv2.IMREAD_UNCHANGED)
# print(img.shape)
# plt.imshow((img*255).astype(np.uint8))